In [ ]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pickle
import pandas as pd 
import math
import numpy as np
import torch

from bayes_opt.utils import test_learning
from sklearn.metrics import f1_score, accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from Models.cslr import cslr

train_sets = pickle.load(open("datasets/train_sets_bank.dat","rb"))
test_sets = pickle.load(open("datasets/test_sets_bank.dat","rb"))
val_sets = [pickle.load(open("datasets/val_sets_bank.dat","rb")), pickle.load(open("datasets/val_test_sets_bank.dat","rb"))]
features = pickle.load(open("datasets/bank_features","rb"))
returns = pickle.load(open("datasets/bank_returns","rb"))
outcomes = pickle.load(open("datasets/bank_outcomes","rb"))
rmax = np.zeros(len(returns))
for i in range(len(returns)):
    rmax[i] = returns[i,np.argmax(returns[i])]
    
replication = 5
n_splits = 10
n_trials = 50
n_epochs = 1000
n_steps = 1000
device = torch.device("cpu")
bank_results = []
max_batchsize = math.floor(math.log2(features.shape[0]))
batch_norm = [False, True]
lr_rates = [0.00005, 0.01]
batch_sizes = [8, max_batchsize]
bayes = True
time_limit = 600
scaler = MinMaxScaler()

for bn in batch_norm:

    for rep in range(replication):

        results = (np.zeros((n_splits,12)) if bn is False else np.zeros((n_splits,4)))
                   
        for i in range(n_splits):

            config = {
                "rep": rep,
                "set": i,
                "bayes": bayes,
                "n_trials": n_trials,
                "max_batch": max_batchsize,
                "lr": lr_rates,
                "batches": batch_sizes,
                "val_sets": val_sets,
                "n_epochs": n_epochs,
                "n_steps": n_steps,
                "batchnorm": bn,
                "device": device,
                "scaler": scaler,
                "time_limit": time_limit
            }
            
            x_train, x_test = features[train_sets[rep][i]], features[test_sets[rep][i]]
            r_train, r_test = returns[train_sets[rep][i]], returns[test_sets[rep][i]]
            rmax_train, rmax_test = rmax[train_sets[rep][i]], rmax[test_sets[rep][i]]           
            model = cslr(config,
                         x_train,
                         r_train,
                         x_test,
                         r_test)
            
            test_probs, train_probs = model.gradient()
            test_return, test_outcome = test_learning(test_probs, r_test)
            train_return, _ = test_learning(train_probs, r_train)
            results[i,0], results[i,1] = test_return/sum(rmax_test), train_return/sum(rmax_train)
            results[i,2] = accuracy_score(outcomes[test_sets[rep][i]], test_outcome)
            results[i,3] = f1_score(outcomes[test_sets[rep][i]], test_outcome)       
            
            if bn is False:
                
                model.give_initial = True
                test_probs, train_probs = model.mip_opt()
                test_return, test_outcome = test_learning(test_probs, r_test)
                train_return, _ = test_learning(train_probs, r_train)
                results[i,4], results[i,5] = test_return/sum(rmax_test), train_return/sum(rmax_train)
                results[i,6] = accuracy_score(outcomes[test_sets[rep][i]], test_outcome)
                results[i,7] = f1_score(outcomes[test_sets[rep][i]], test_outcome)    

                model.give_initial = False
                test_probs, train_probs = model.mip_opt()
                test_return, test_outcome = test_learning(test_probs, r_test)
                train_return, _ = test_learning(train_probs, r_train)
                results[i,8], results[i,9] = test_return/sum(rmax_test), train_return/sum(rmax_train)
                results[i,10] = accuracy_score(outcomes[test_sets[rep][i]], test_outcome)
                results[i,11] = f1_score(outcomes[test_sets[rep][i]], test_outcome)       
                
        bank_results.append(results)    
        